# Data Preprocessing

In [1]:
import polars as pl
import numpy as np

## Loading data

In [2]:
lter = pl.read_csv("./../data/penguins_lter.csv",try_parse_dates=True)

In [3]:
lter = (
    lter
    # ---Parse Date Egg into datetime, keep month-year ---
    .with_columns(
        pl.col("Date Egg").str.strptime(pl.Date, "%m/%d/%y", strict=False).alias("egg_date")
    )
    # Add formatted date + year
    .with_columns([
        pl.col("egg_date").dt.strftime("%b-%Y").alias("Date egg"),
        pl.col("egg_date").dt.year().alias("year")
    ])
    # --- Convert studyName → Study years ---
    .with_columns(
        pl.when(pl.col("studyName").str.contains(r"^PAL\d{4}$"))
        .then(
            pl.concat_str([
                (2000 + pl.col("studyName").str.slice(3, 2).cast(pl.Int64)).cast(pl.Utf8),
                pl.lit("-"),
                pl.col("studyName").str.slice(5, 2)
            ])
        )
        .otherwise(None)
        .alias("Study years")
    )
)


# --- Rename columns to final schema ---
lter = lter.rename({
    "Species": "species",
    "Region": "Region",
    "Island": "Island",
    "Clutch Completion": "Clutch Completion",
    "Culmen Length (mm)": "culmen length",
    "Culmen Depth (mm)": "culmen depth",
    "Flipper Length (mm)": "flipper length",
    "Body Mass (g)": "body mass",
    "Sex": "sex",
    "Delta 15 N (o/oo)": "Delta 15 N",
    "Delta 13 C (o/oo)": "Delta 13",
    "Comments": "comments"
})

# --- Species normalization ---
species_map = {
    "adelie": "Adelie",
    "chinstrap": "Chinstrap",
    "gentoo": "Gentoo"
}

lter = lter.with_columns(
    pl.col("species")
    .str.to_lowercase()
    .map_elements(lambda x: next((v for k, v in species_map.items() if k in x), x))
    .alias("species")
)

# --- Sex normalization ---
sex_map = {
    "male": "Male",
    "female": "Female"
}

lter = lter.with_columns(
    pl.col("sex")
    .str.to_lowercase()
    .map_elements(lambda x: sex_map.get(x, None))
    .alias("sex")
)


# --- Cast values as numeric(float) ---
for col in ["culmen length", "culmen depth", "flipper length", "body mass",
            "Delta 15 N", "Delta 13"]:
    lter = lter.with_columns(pl.col(col).cast(pl.Float64, strict=False))

# --- Select only the required columns ---
lter_clean = lter.select([
    "Study years", "species", "Region", "Island", "Clutch Completion",
    "Date egg", "culmen length", "culmen depth", "flipper length",
    "body mass", "sex", "Delta 15 N", "Delta 13", "comments", "year" 
])

In [4]:
print(lter_clean.null_count())

shape: (1, 15)
┌─────────────┬─────────┬────────┬────────┬───┬────────────┬──────────┬──────────┬──────┐
│ Study years ┆ species ┆ Region ┆ Island ┆ … ┆ Delta 15 N ┆ Delta 13 ┆ comments ┆ year │
│ ---         ┆ ---     ┆ ---    ┆ ---    ┆   ┆ ---        ┆ ---      ┆ ---      ┆ ---  │
│ u32         ┆ u32     ┆ u32    ┆ u32    ┆   ┆ u32        ┆ u32      ┆ u32      ┆ u32  │
╞═════════════╪═════════╪════════╪════════╪═══╪════════════╪══════════╪══════════╪══════╡
│ 0           ┆ 0       ┆ 0      ┆ 0      ┆ … ┆ 14         ┆ 13       ┆ 318      ┆ 0    │
└─────────────┴─────────┴────────┴────────┴───┴────────────┴──────────┴──────────┴──────┘


In [5]:
df = lter_clean.clone()

for col in df.columns:
    if df.schema[col] == pl.Float64:
        # Mean and std
        mean_val = df.select(pl.col(col).mean()).item()
        std_val = df.select(pl.col(col).std()).item()

        if mean_val is None or std_val is None:
            continue

        lower, upper = mean_val - std_val, mean_val + std_val

        # Get mask of nulls
        mask = df.select(pl.col(col).is_null()).to_series().to_numpy()

        if mask.sum() > 0:  # only if nulls exist
            # Generate random values for those nulls
            random_fill = np.random.uniform(lower, upper, mask.sum())

            # Convert column to numpy, replace nulls manually
            col_vals = df[col].to_numpy()
            col_vals[mask] = random_fill

            # Put back into df
            df = df.with_columns(pl.Series(name=col, values=col_vals))
            

In [6]:
print(df.null_count())

shape: (1, 15)
┌─────────────┬─────────┬────────┬────────┬───┬────────────┬──────────┬──────────┬──────┐
│ Study years ┆ species ┆ Region ┆ Island ┆ … ┆ Delta 15 N ┆ Delta 13 ┆ comments ┆ year │
│ ---         ┆ ---     ┆ ---    ┆ ---    ┆   ┆ ---        ┆ ---      ┆ ---      ┆ ---  │
│ u32         ┆ u32     ┆ u32    ┆ u32    ┆   ┆ u32        ┆ u32      ┆ u32      ┆ u32  │
╞═════════════╪═════════╪════════╪════════╪═══╪════════════╪══════════╪══════════╪══════╡
│ 0           ┆ 0       ┆ 0      ┆ 0      ┆ … ┆ 0          ┆ 0        ┆ 318      ┆ 0    │
└─────────────┴─────────┴────────┴────────┴───┴────────────┴──────────┴──────────┴──────┘


In [7]:
df.write_csv("./../data/cleaned_penguin.csv")